In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import math

In [0]:
input_size = 3072
hidden_size = 400
neck_size = 128
num_classes = 784
num_epochs = 10
batch_size = 100
iterations = 10
learning_rate = 0.001

In [4]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = dsets.CIFAR10(
    root='./data',
    train=True,
    transform=trans,
    download=True
)

test_dataset = dsets.CIFAR10(
    root='./data',
    train=False,
    transform=trans
)


Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [0]:
class PerceptrAE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, neck_size)
        # Decoder
        self.fc4 = nn.Linear(neck_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, input_size)
    
    def encode(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return x
    
    def decode(self, x):
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc6(x))
        return x
    
    def forward(self, x):
        out = self.encode(x)
        out = self.decode(out)
        return out

In [0]:
class ConvAE(nn.Module):
    def __init__(self):
        super(ConvAE, self).__init__()
        # Encoder
        self.layer1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(16, 8, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        # Decoder
        self.layer4 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.Upsample(scale_factor=2, mode='bilinear'))
        self.layer5 = nn.Sequential(nn.Conv2d(8, 16, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.Upsample(scale_factor=2, mode='bilinear'))
        self.layer6 = nn.Sequential(nn.Conv2d(16, 3, kernel_size=5, stride=1, padding=2), 
                                    nn.Sigmoid(), nn.Upsample(scale_factor=2, mode='bilinear'))
    
    def encode(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out) 
        return out
    
    def decode(self, x):
        out = self.layer4(x)
        out = self.layer5(out)
        out = self.layer6(out) 
        return out
    
    def forward(self, x):
        out = self.encode(x)
        out = self.decode(out)
        return out

In [65]:
%%time
autoencoder = ConvAE().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate)

total_step = len(train_loader)
loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
      for i, (image, labels) in enumerate(train_loader):
            image = image.to(torch.device('cuda'))

            out_image = autoencoder(image)
            loss = criterion(out_image, image)
            loss_list.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % batch_size == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                    .format(epoch + 1, num_epochs, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Epoch [1/10], Step [0/500], Loss: 1.1890079975128174
Epoch [1/10], Step [100/500], Loss: 0.606131911277771
Epoch [1/10], Step [200/500], Loss: 0.641532301902771
Epoch [1/10], Step [300/500], Loss: 0.6819426417350769
Epoch [1/10], Step [400/500], Loss: 0.5779098868370056
Epoch [2/10], Step [0/500], Loss: 0.7202984690666199
Epoch [2/10], Step [100/500], Loss: 0.6899333000183105
Epoch [2/10], Step [200/500], Loss: 0.6573451161384583
Epoch [2/10], Step [300/500], Loss: 0.6309413909912109
Epoch [2/10], Step [400/500], Loss: 0.6087682843208313
Epoch [3/10], Step [0/500], Loss: 0.5527730584144592
Epoch [3/10], Step [100/500], Loss: 0.6258824467658997
Epoch [3/10], Step [200/500], Loss: 0.5422277450561523
Epoch [3/10], Step [300/500], Loss: 0.594713032245636
Epoch [3/10], Step [400/500], Loss: 0.5183201432228088
Epoch [4/10], Step [0/500], Loss: 0.5123148560523987
Epoch [4/10], Step [100/500], Loss: 0.6159372329711914
Epoch [4/10], Step [200/500], Loss: 0.5803899765014648
Epoch [4/10], Step [3

In [68]:
for epoch in range(num_epochs):
    for i, (image, labels) in enumerate(train_loader):
        image = image.to(torch.device('cuda'))
  
        out_image = autoencoder(image)
        loss = criterion(out_image, image)
        loss_list.append(loss.item())
      
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
        if i % batch_size == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Epoch [1/10], Step [0/500], Loss: 0.4696296155452728
Epoch [1/10], Step [100/500], Loss: 0.6005987524986267
Epoch [1/10], Step [200/500], Loss: 0.5246269702911377
Epoch [1/10], Step [300/500], Loss: 0.5158776044845581
Epoch [1/10], Step [400/500], Loss: 0.4770846664905548
Epoch [2/10], Step [0/500], Loss: 0.5511453151702881
Epoch [2/10], Step [100/500], Loss: 0.4556095600128174
Epoch [2/10], Step [200/500], Loss: 0.4395073652267456
Epoch [2/10], Step [300/500], Loss: 0.5440658926963806
Epoch [2/10], Step [400/500], Loss: 0.5001341104507446
Epoch [3/10], Step [0/500], Loss: 0.5373076796531677
Epoch [3/10], Step [100/500], Loss: 0.5253438353538513
Epoch [3/10], Step [200/500], Loss: 0.6252644062042236
Epoch [3/10], Step [300/500], Loss: 0.4697343707084656
Epoch [3/10], Step [400/500], Loss: 0.5373998284339905
Epoch [4/10], Step [0/500], Loss: 0.4938448667526245
Epoch [4/10], Step [100/500], Loss: 0.5330120325088501
Epoch [4/10], Step [200/500], Loss: 0.532616913318634
Epoch [4/10], Step 

In [0]:
# baseline
class ConvClassyfier(nn.Module): 
    def __init__(self): 
        super(ConvClassyfier, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=1), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2))  
        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(256, 512) 
        self.fc2 = nn.Linear(512, 374)
        self.fc3 = nn.Linear(374, 10)
    
    def forward(self, x):
        out = x.reshape(x.size(0), -1)  
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out) 
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out)) 
        out = F.log_softmax(self.fc3(out)) 
        return out

In [113]:
%%time
model = ConvClassyfier().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

total_step = len(train_loader)
loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
   for i, (image, labels) in enumerate(train_loader):
     image = image.to(torch.device('cuda'))
     labels = labels.to(torch.device('cuda'))
     # image, labels = image.view(-1, 32, 32), labels.view(-1, 10)

     outputs = model(image)
     loss = criterion(outputs, labels)
     loss_list.append(loss.item())

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if i % batch_size == 0:
         print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
             .format(epoch + 1, num_epochs, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/10], Step [0/500], Loss: 2.298551082611084
Epoch [1/10], Step [100/500], Loss: 1.789108157157898
Epoch [1/10], Step [200/500], Loss: 1.6721948385238647
Epoch [1/10], Step [300/500], Loss: 1.4951080083847046
Epoch [1/10], Step [400/500], Loss: 1.4783755540847778
Epoch [2/10], Step [0/500], Loss: 1.4158830642700195
Epoch [2/10], Step [100/500], Loss: 1.2945237159729004
Epoch [2/10], Step [200/500], Loss: 1.2300673723220825
Epoch [2/10], Step [300/500], Loss: 1.3615756034851074
Epoch [2/10], Step [400/500], Loss: 1.132836103439331
Epoch [3/10], Step [0/500], Loss: 1.1295536756515503
Epoch [3/10], Step [100/500], Loss: 0.9369361996650696
Epoch [3/10], Step [200/500], Loss: 1.1858471632003784
Epoch [3/10], Step [300/500], Loss: 0.9874433875083923
Epoch [3/10], Step [400/500], Loss: 0.9776670336723328
Epoch [4/10], Step [0/500], Loss: 0.9711243510246277
Epoch [4/10], Step [100/500], Loss: 0.4742954969406128
Epoch [4/10], Step [200/500], Loss: 0.9344733357429504
Epoch [4/10], Step [3

In [116]:
for epoch in range(num_epochs):
     for i, (image, labels) in enumerate(train_loader):
        image = image.to(torch.device('cuda'))
        labels = labels.to(torch.device('cuda'))

        outputs = model(image)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % batch_size == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                .format(epoch + 1, num_epochs, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/10], Step [0/500], Loss: 0.3321221172809601
Epoch [1/10], Step [100/500], Loss: 0.5408867001533508
Epoch [1/10], Step [200/500], Loss: 0.3831905722618103
Epoch [1/10], Step [300/500], Loss: 0.47524967789649963
Epoch [1/10], Step [400/500], Loss: 0.3054683804512024
Epoch [2/10], Step [0/500], Loss: 0.29435834288597107
Epoch [2/10], Step [100/500], Loss: 0.17314183712005615
Epoch [2/10], Step [200/500], Loss: 0.2673536539077759
Epoch [2/10], Step [300/500], Loss: 0.2401295155286789
Epoch [2/10], Step [400/500], Loss: 0.21355368196964264
Epoch [3/10], Step [0/500], Loss: 0.28339818120002747
Epoch [3/10], Step [100/500], Loss: 0.12111752480268478
Epoch [3/10], Step [200/500], Loss: 0.2966020703315735
Epoch [3/10], Step [300/500], Loss: 0.4660169184207916
Epoch [3/10], Step [400/500], Loss: 0.31218981742858887
Epoch [4/10], Step [0/500], Loss: 0.2487531304359436
Epoch [4/10], Step [100/500], Loss: 0.22194026410579681
Epoch [4/10], Step [200/500], Loss: 0.4839268624782562
Epoch [4/1

In [0]:
class DE(nn.Module):
    def __init__(self):
        super(DE, self).__init__()
        # Encoder 
        self.fc1 = nn.Linear(input_size * 2, neck_size)
        # Decoder
        self.fc2 = nn.Linear(neck_size + input_size, input_size)

        def encode(self, x, y):
            x = F.relu(self.fc1(x, y))
            return x

        def decode(self, x, y):
            x = torch.sigmoid(self.fc2(x, y))
            return x

        def forward(self, x, y):
            out = self.encode(x, y)
            out = self.decode(out, y)
            return out

In [117]:
%%time
total = 0
correct = 0

for i, (image, labels) in enumerate(test_loader):
  # image = image.view(-1, input_size)
  image = image.to(torch.device('cuda'))
  labels = labels.to(torch.device('cuda'))
  # image, labels = Variable(image), Variable(labels)
  # outputs = autoencoder(image)
  # outputs, labels = Variable(outputs), Variable(labels)
  outputs = model(image)
  _, predicted = torch.max(outputs.data, 1)  # Выбор лучшего класса из выходных данных: класс с лучшим счетом
  total += labels.size(0)                    # Увеличиваем суммарный счет
  correct += (predicted == labels).sum()     # Увеличиваем корректный счет
     
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10K test images: 72 %
CPU times: user 1.58 s, sys: 8.93 ms, total: 1.59 s
Wall time: 1.59 s
